In [1]:
%load_ext dotenv
%dotenv

In [2]:
import os
import base64
import httpx
from typing import Dict, Any, Optional

# Get the API key from environment variables
API_KEY = os.getenv("COMPANIES_HOUSE_API_KEY")
BASE_URL = "https://api.company-information.service.gov.uk"


def search_companies(query: str) -> Dict[str, Any]:
    """
    Search for companies by name or number.

    Args:
        query: The search term to find companies

    Returns:
        Dict containing search results
    """
    if not API_KEY:
        raise ValueError("COMPANIES_HOUSE_API_KEY environment variable not set")

    # Companies House API requires Basic Auth with API key as username and empty password
    auth_str = f"{API_KEY}:"
    auth_bytes = auth_str.encode("ascii")
    auth_b64 = base64.b64encode(auth_bytes).decode("ascii")

    headers = {"Authorization": f"Basic {auth_b64}", "Accept": "application/json"}

    url = f"{BASE_URL}/search/companies"
    params = {"q": query}

    try:
        with httpx.Client() as client:
            response = client.get(url, headers=headers, params=params)
            response.raise_for_status()
            return response.json()
    except httpx.HTTPStatusError as e:
        print(f"HTTP Error: {e}")
        return {"error": str(e)}
    except Exception as e:
        print(f"Error: {e}")
        return {"error": str(e)}


def get_company_profile(company_number: str) -> Dict[str, Any]:
    """
    Get detailed company information by company number.

    Args:
        company_number: The Companies House company number

    Returns:
        Dict containing company information
    """
    if not API_KEY:
        raise ValueError("COMPANIES_HOUSE_API_KEY environment variable not set")

    # Companies House API requires Basic Auth with API key as username and empty password
    auth_str = f"{API_KEY}:"
    auth_bytes = auth_str.encode("ascii")
    auth_b64 = base64.b64encode(auth_bytes).decode("ascii")

    headers = {"Authorization": f"Basic {auth_b64}", "Accept": "application/json"}

    url = f"{BASE_URL}/company/{company_number}"

    try:
        with httpx.Client() as client:
            response = client.get(url, headers=headers)
            response.raise_for_status()
            return response.json()
    except httpx.HTTPStatusError as e:
        print(f"HTTP Error: {e}")
        return {"error": str(e)}
    except Exception as e:
        print(f"Error: {e}")
        return {"error": str(e)}

In [3]:
# Example: Search for a company
company_name = "Tesla"
print(f"Searching for companies matching '{company_name}'...")
search_results = search_companies(company_name)

# Print first few results
if "items" in search_results and search_results["items"]:
    print(f"Found {len(search_results['items'])} results. Here are the first few:")
    for i, company in enumerate(search_results["items"][:3]):
        print(f"{i + 1}. {company.get('title')} - Number: {company.get('company_number')}")

    # Get detailed information about the first company
    first_company_number = search_results["items"][0]["company_number"]
    print(f"\nGetting detailed information for company number {first_company_number}...")
    company_details = get_company_profile(first_company_number)

    # Print some key details
    print(f"Company Name: {company_details.get('company_name')}")
    print(f"Company Status: {company_details.get('company_status')}")
    print(f"Incorporation Date: {company_details.get('date_of_creation')}")

    address = company_details.get("registered_office_address", {})
    address_str = ", ".join([v for k, v in address.items() if v])
    print(f"Registered Address: {address_str}")
else:
    print("No companies found or there was an error.")

Searching for companies matching 'Tesla'...
Found 20 results. Here are the first few:
1. TESLA LIMITED - Number: 03058989
2. TESLA MAGNETIC LIMITED - Number: 03698072
3. HUNTSMAN CARS LTD - Number: 15098099

Getting detailed information for company number 03058989...
Company Name: TESLA LIMITED
Company Status: active
Incorporation Date: 1995-05-19
Registered Address: Unit 1, Water Lane Industrial Estate, Storrington, RH20 3EA, West Sussex
